In [1]:
!pip install transformers==4.6.1

     |████████████████████████████████| 2.2 MB 12.9 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 


In [2]:
# for the SMS data
import pandas as pd
df = pd.read_csv('/content/SMSSpamCollection',sep='\t',names=["label","message"])

In [3]:
'''
#for the Twitter data
import pandas as pd
df = pd.read_csv('/content/Twitter.csv')  #,sep='\t',names=["label","message"]
df.drop(['Id','following','followers','actions','is_retweet','location'],axis=1,inplace=True)
df.rename(columns = {'Tweet':'message','Type':'label'},inplace=True)
'''

'\n#for the Twitter data\nimport pandas as pd\ndf = pd.read_csv(\'/content/train.csv\')  #,sep=\'\t\',names=["label","message"]\ndf.drop([\'Id\',\'following\',\'followers\',\'actions\',\'is_retweet\',\'location\'],axis=1,inplace=True)\ndf.rename(columns = {\'Tweet\':\'message\',\'Type\':\'label\'},inplace=True)\n'

In [4]:
df.head(100)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
95,spam,Your free ringtone is waiting to be collected....
96,ham,Watching telugu movie..wat abt u?
97,ham,i see. When we finish we have loads of loans t...
98,ham,Hi. Wk been ok - on hols now! Yes on for a bit...


In [5]:
len(df)

5572

In [6]:
x = list(df['message'])

In [7]:
y = list(df['label'])

In [8]:
#for the SMS data
y = list(pd.get_dummies(y,drop_first=True)['spam']) 

#for the Twitter data
#y = list(pd.get_dummies(y,drop_first=True)['Spam']) 

In [9]:
y

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [11]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [12]:
train_encodings = tokenizer(x_train, truncation=True, padding=True)

In [13]:
test_encodings = tokenizer(x_test, truncation=True, padding=True)

In [14]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [15]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',    
    num_train_epochs=3,             
    per_device_train_batch_size=4,   
    per_device_eval_batch_size=8,    
    warmup_steps=500,         
    weight_decay=0.01,       
    logging_dir='./logs', 
    logging_steps=10,
)

In [16]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,             
    args=training_args,              
    train_dataset=train_dataset,     
    eval_dataset=test_dataset     
)

trainer.train()

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [17]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.02833584717341832}

In [18]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.4347916, -3.548484 ],
       [-2.5355961,  3.2052326],
       [ 3.4502776, -3.5752635],
       ...,
       [ 3.4557784, -3.5803087],
       [-2.4865565,  3.152561 ],
       [ 3.4133086, -3.5537417]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.028337795393807548})

In [19]:
pred = trainer.predict(test_dataset=test_dataset).predictions
#prob = softmax(pred, axis = 1)
#pred_label = pred.argmax(-1)

In [20]:
pred

array([[ 3.4347916, -3.548484 ],
       [-2.5355961,  3.2052326],
       [ 3.4502776, -3.5752635],
       ...,
       [ 3.4557784, -3.5803087],
       [-2.4865565,  3.152561 ],
       [ 3.4133086, -3.5537417]], dtype=float32)

In [21]:
pred_label = pred.argmax(-1)

In [22]:
#list(pred_label)

In [23]:
#y_test

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_label)

0.9937219730941704

In [25]:
from sklearn.metrics import confusion_matrix
confusionMatrixResult=confusion_matrix(y_test,pred_label)
confusionMatrixResult

array([[955,   0],
       [  7, 153]])